In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import json
import pandas as pd
import sys
from tqdm import tqdm_notebook as tqdm
import copy
import os.path as osp
%run ./plot_functions.ipynb
sys.path.insert(0, '../src/')


output_path =  ../output/figures
plt = single_column_plt_params(plt)
plt = double_column_plt_params(plt)


In [2]:
from benchmark_utilities import  benchmark_pnml, benchmark_odin, compute_regret
plt = single_column_plt_params(plt)


## Densnet CIFAR100

In [3]:
base_dir = osp.join('..', 'output', 'embeddings')
pnml_datasets_cifar100 = {
    'train_labels': 'cifar100_train_labels.npy',
    'trainset':  'densenet_cifar100_cifar100_train_pnml.npy',
    'testset': 'densenet_cifar100_cifar100_test_pnml.npy',
    'Imagenet (crop)': 'densenet_cifar100_Imagenet_test_pnml.npy',
    'Imagenet (resize)': 'densenet_cifar100_Imagenet_resize_test_pnml.npy',
    'LSUN (crop)': 'densenet_cifar100_LSUN_test_pnml.npy',
    'LSUN (resize)': 'densenet_cifar100_LSUN_resize_test_pnml.npy',
    'iSUN': 'densenet_cifar100_iSUN_test_pnml.npy'
}

odin_datasets_cifar100 = {
    'testset': 'densenet_cifar100_cifar100_test_odin.npy',
    'Imagenet (crop)': 'densenet_cifar100_Imagenet_test_odin.npy',
    'Imagenet (resize)': 'densenet_cifar100_Imagenet_resize_test_odin.npy',
    'LSUN (crop)': 'densenet_cifar100_LSUN_test_odin.npy',
    'LSUN (resize)': 'densenet_cifar100_LSUN_resize_test_odin.npy',
    'iSUN': 'densenet_cifar100_iSUN_test_odin.npy'
}

pnml_datasets_cifar100 = {key: osp.join(base_dir, value)
                          for key, value in pnml_datasets_cifar100.items()}
odin_datasets_cifar100 = {key: osp.join(base_dir, value)
                          for key, value in odin_datasets_cifar100.items()}


In [4]:
pnml_df_cifar100 = benchmark_pnml(pnml_datasets_cifar100).round(1)
print('pNML CIFAR100')
pnml_df_cifar100

pNML for Imagenet (crop)
pNML for Imagenet (resize)
pNML for LSUN (crop)
pNML for LSUN (resize)
pNML for iSUN
pNML CIFAR100


,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
Imagenet (crop),pNML,29.4,17.2,92.8,92.2,93.4
Imagenet (resize),pNML,15.2,10.1,96.7,96.4,96.9
LSUN (crop),pNML,80.7,42.9,67.6,69.3,68.0
LSUN (resize),pNML,10.6,7.8,97.9,97.9,97.8
iSUN,pNML,12.3,8.6,97.5,97.5,97.6


In [5]:
odin_df_cifar100 = benchmark_odin(odin_datasets_cifar100).round(1)
print('ODIN CIFAR100')
odin_df_cifar100

ODIN for Imagenet (crop)
ODIN for Imagenet (resize)
ODIN for LSUN (crop)
ODIN for LSUN (resize)
ODIN for iSUN
ODIN CIFAR100


,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
Imagenet (crop),ODIN,28.7,16.8,94.1,94.4,93.8
Imagenet (resize),ODIN,57.8,31.4,85.1,85.6,84.0
LSUN (crop),ODIN,17.2,11.1,96.7,96.9,96.5
LSUN (resize),ODIN,60.6,32.8,85.3,86.4,83.6
iSUN,ODIN,63.8,34.4,84.0,85.1,81.9


In [6]:
result_cifar100 = pd.concat(
    [odin_df_cifar100, pnml_df_cifar100]).sort_index(kind='merge')
result_cifar100


,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
Imagenet (crop),ODIN,28.7,16.8,94.1,94.4,93.8
Imagenet (crop),pNML,29.4,17.2,92.8,92.2,93.4
Imagenet (resize),ODIN,57.8,31.4,85.1,85.6,84.0
Imagenet (resize),pNML,15.2,10.1,96.7,96.4,96.9
LSUN (crop),ODIN,17.2,11.1,96.7,96.9,96.5
LSUN (crop),pNML,80.7,42.9,67.6,69.3,68.0
LSUN (resize),ODIN,60.6,32.8,85.3,86.4,83.6
LSUN (resize),pNML,10.6,7.8,97.9,97.9,97.8
iSUN,ODIN,63.8,34.4,84.0,85.1,81.9
iSUN,pNML,12.3,8.6,97.5,97.5,97.6


## Densnet CIFAR10

In [7]:
base_dir = osp.join('..', 'output', 'embeddings')
pnml_datasets_cifar10 = {
    'train_labels': 'cifar10_train_labels.npy',
    'trainset':  'densenet_cifar10_cifar10_train_pnml.npy',
    'testset': 'densenet_cifar10_cifar10_test_pnml.npy',
    'Imagenet (crop)': 'densenet_cifar10_Imagenet_test_pnml.npy',
    'Imagenet (resize)':    'densenet_cifar10_Imagenet_resize_test_pnml.npy',
    'LSUN (crop)': 'densenet_cifar10_LSUN_test_pnml.npy',
    'LSUN (resize)': 'densenet_cifar10_LSUN_resize_test_pnml.npy',
    'iSUN': 'densenet_cifar10_iSUN_test_pnml.npy'
}

odin_datasets_cifar10 = {
    'testset': 'densenet_cifar10_cifar10_test_odin.npy',
    'Imagenet (crop)': 'densenet_cifar10_Imagenet_test_odin.npy',
    'Imagenet (resize)':    'densenet_cifar10_Imagenet_resize_test_odin.npy',
    'LSUN (crop)': 'densenet_cifar10_LSUN_test_odin.npy',
    'LSUN (resize)': 'densenet_cifar10_LSUN_resize_test_odin.npy',
    'iSUN': 'densenet_cifar10_iSUN_test_odin.npy'
}

pnml_datasets_cifar10 = {key: osp.join(base_dir, value)
                         for key, value in pnml_datasets_cifar10.items()}
odin_datasets_cifar10 = {key: osp.join(base_dir, value)
                         for key, value in odin_datasets_cifar10.items()}


In [8]:
regret_df_cifar10 = benchmark_pnml(pnml_datasets_cifar10).round(1)
print('pNML CIFAR10')
regret_df_cifar10

pNML for Imagenet (crop)
pNML for Imagenet (resize)
pNML for LSUN (crop)
pNML for LSUN (resize)
pNML for iSUN
pNML CIFAR10


,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
Imagenet (crop),pNML,13.7,9.3,97.4,97.4,97.6
Imagenet (resize),pNML,15.8,10.4,96.7,96.6,96.9
LSUN (crop),pNML,28.8,16.9,94.3,94.5,94.5
LSUN (resize),pNML,10.6,7.8,98.0,98.1,97.8
iSUN,pNML,12.3,8.6,97.6,97.6,97.6


In [9]:
odin_df_cifar10 = benchmark_odin(odin_datasets_cifar10).round(1)
print('ODIN CIFAR10')
odin_df_cifar10

ODIN for Imagenet (crop)
ODIN for Imagenet (resize)
ODIN for LSUN (crop)
ODIN for LSUN (resize)
ODIN for iSUN
ODIN CIFAR10


,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
Imagenet (crop),ODIN,4.2,4.6,99.1,99.2,99.1
Imagenet (resize),ODIN,7.4,6.2,98.5,98.6,98.5
LSUN (crop),ODIN,11.3,8.2,97.9,98.1,97.9
LSUN (resize),ODIN,3.8,4.4,99.2,99.3,99.2
iSUN,ODIN,6.0,5.5,98.9,98.9,98.9


In [10]:
result_cifar10 = pd.concat([odin_df_cifar10, regret_df_cifar10]).sort_index(kind='merge')
result_cifar10

,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
Imagenet (crop),ODIN,4.2,4.6,99.1,99.2,99.1
Imagenet (crop),pNML,13.7,9.3,97.4,97.4,97.6
Imagenet (resize),ODIN,7.4,6.2,98.5,98.6,98.5
Imagenet (resize),pNML,15.8,10.4,96.7,96.6,96.9
LSUN (crop),ODIN,11.3,8.2,97.9,98.1,97.9
LSUN (crop),pNML,28.8,16.9,94.3,94.5,94.5
LSUN (resize),ODIN,3.8,4.4,99.2,99.3,99.2
LSUN (resize),pNML,10.6,7.8,98.0,98.1,97.8
iSUN,ODIN,6.0,5.5,98.9,98.9,98.9
iSUN,pNML,12.3,8.6,97.6,97.6,97.6
